# Aggregation and Grouping

>`groupby`

In [68]:
import numpy as np
import pandas as pd

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## Planets data

Contains information on extrasolar planets

In [69]:
import seaborn as sns 
planets = sns.load_dataset('planets')
planets.shape

(1035, 6)

In [70]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [71]:
planets.index

RangeIndex(start=0, stop=1035, step=1)

In [72]:
rng = np.random.default_rng(seed=42)

## Simple Series Aggregation

In [73]:
ser = pd.Series(rng.random(5))
ser

0    0.773956
1    0.438878
2    0.858598
3    0.697368
4    0.094177
dtype: float64

In [74]:
ser.sum(), ser.mean()

(2.8629777851664118, 0.5725955570332824)

## Simple DataFrame Aggregation

In [75]:
df = pd.DataFrame({'A': rng.random(5),
                   'B': rng.random(5)})
df

,A,B
0,0.975622,0.370798
1,0.761140,0.926765
2,0.786064,0.643865
3,0.128114,0.822762
4,0.450386,0.443414


In [76]:
df.mean(), df.mean(axis='columns')

(A    0.620265
 B    0.641521
 dtype: float64,
 0    0.673210
 1    0.843952
 2    0.714965
 3    0.475438
 4    0.446900
 dtype: float64)

In [77]:
df.mean(axis=1)

0    0.673210
1    0.843952
2    0.714965
3    0.475438
4    0.446900
dtype: float64

## Describe

In [78]:
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


## `groupby`: Split, Apply, Combine

### Example

<img src="./images/03.08-split-apply-combine.png" />

In [79]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [80]:
df.groupby('key')

In [81]:
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


# The `GroupBy` Object

## Column Indexing

> DataFrameGroupBy

In [82]:
planets.groupby('method')

>Series GroupBy

In [83]:
planets.groupby('method')['orbital_period']

In [84]:
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

## Iteration

>Slower than `apply()`

In [85]:
for (method, group) in planets.groupby('method'): 
    print(f"{method:<30} {group.shape=}")

Astrometry                     group.shape=(2, 6)
Eclipse Timing Variations      group.shape=(9, 6)
Imaging                        group.shape=(38, 6)
Microlensing                   group.shape=(23, 6)
Orbital Brightness Modulation  group.shape=(3, 6)
Pulsar Timing                  group.shape=(5, 6)
Pulsation Timing Variations    group.shape=(1, 6)
Radial Velocity                group.shape=(553, 6)
Transit                        group.shape=(397, 6)
Transit Timing Variations      group.shape=(4, 6)


## Dispatch

In [86]:
planets.groupby('method')['year'].describe()

,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
Astrometry,2.0,2011.500000,2.121320,2010.0,2010.75,2011.5,2012.25,2013.0
Eclipse Timing Variations,9.0,2010.000000,1.414214,2008.0,2009.00,2010.0,2011.00,2012.0
Imaging,38.0,2009.131579,2.781901,2004.0,2008.00,2009.0,2011.00,2013.0
Microlensing,23.0,2009.782609,2.859697,2004.0,2008.00,2010.0,2012.00,2013.0
Orbital Brightness Modulation,3.0,2011.666667,1.154701,2011.0,2011.00,2011.0,2012.00,2013.0
Pulsar Timing,5.0,1998.400000,8.384510,1992.0,1992.00,1994.0,2003.00,2011.0
Pulsation Timing Variations,1.0,2007.000000,NaN,2007.0,2007.00,2007.0,2007.00,2007.0
Radial Velocity,553.0,2007.518987,4.249052,1989.0,2005.00,2009.0,2011.00,2014.0
Transit,397.0,2011.236776,2.077867,2002.0,2010.00,2012.0,2013.00,2014.0


## Aggregate, Filter, Transform, Apply

In [87]:
rng = np.random.default_rng(seed=0)

In [88]:
df = pd.DataFrame({
    'key': ['A', 'B', 'C', 'A', 'B', 'C'],
    'data1': range(6),
    'data2': rng.integers(0, 10, 6)
    }, columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,8
1,B,1,6
2,C,2,5
3,A,3,2
4,B,4,3
5,C,5,0


## Aggregation

In [89]:
df.groupby('key').aggregate(['min', np.median, max])

/tmp/ipykernel_108862/968873422.py:1: FutureWarning: The provided callable <function median at 0x7f2da005e0c0> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  df.groupby('key').aggregate(['min', np.median, max])
/tmp/ipykernel_108862/968873422.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df.groupby('key').aggregate(['min', np.median, max])


data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     2    5.0   8
B       1    2.5   4     3    4.5   6
C       2    3.5   5     0    2.5   5

In [90]:
df.groupby('key').aggregate(['min', 'median', 'max'])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     2    5.0   8
B       1    2.5   4     3    4.5   6
C       2    3.5   5     0    2.5   5

In [91]:
df.groupby('key').aggregate({'data1': 'min', 'data2': 'max'})

,data1,data2
key,,
A,0,8
B,1,6
C,2,5


## Filtering

In [92]:
def filter_func(x): 
    return x['data2'].std() > 5

In [93]:
display('df', "df.groupby('key').std()",
        "df.groupby('key').filter(filter_func)")

df
  key  data1  data2
0   A      0      8
1   B      1      6
2   C      2      5
3   A      3      2
4   B      4      3
5   C      5      0

df.groupby('key').std()
       data1     data2
key                   
A    2.12132  4.242641
B    2.12132  2.121320
C    2.12132  3.535534

df.groupby('key').filter(filter_func)
Empty DataFrame
Columns: [key, data1, data2]
Index: []

## Transformation

In [94]:
def center(x): 
    return x - x.mean()
df.groupby('key').transform(center)

,data1,data2
0,-1.5,3.0
1,-1.5,1.5
2,-1.5,2.5
3,1.5,-3.0
4,1.5,-1.5
5,1.5,-2.5


## Apply

In [95]:
def norm_by_data2(x): 
    x['data1'] /= x['data2'].sum()
    return x

df.groupby('key').apply(norm_by_data2)

/tmp/ipykernel_108862/3919926901.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby('key').apply(norm_by_data2)


key     data1  data2
key                       
A   0   A  0.000000      8
    3   A  0.300000      2
B   1   B  0.111111      6
    4   B  0.444444      3
C   2   C  0.400000      5
    5   C  1.000000      0

In [96]:
df.groupby('key').apply(norm_by_data2, include_groups=False)

data1  data2
key                   
A   0  0.000000      8
    3  0.300000      2
B   1  0.111111      6
    4  0.444444      3
C   2  0.400000      5
    5  1.000000      0

# Specify the split key

> with list, array, series or index

In [97]:
L = [0, 1, 0, 1, 2, 0]
df.groupby(L).sum()

,key,data1,data2
0,ACC,7,13
1,BA,4,8
2,B,4,3


In [98]:
df.groupby(df['key']).sum()

,data1,data2
key,,
A,3,10
B,5,9
C,7,5


>With dictionary or series mapping index

In [99]:
df2 = df.set_index('key')
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}
display('df2', 'df2.groupby(mapping).sum()')

,data1,data2
key,,
A,0,8
B,1,6
C,2,5
A,3,2
B,4,3
C,5,0
,data1,data2
key,,
consonant,12,14


## Any Python function

In [100]:
df2.groupby(str.lower).mean()

,data1,data2
key,,
a,1.5,5.0
b,2.5,4.5
c,3.5,2.5


## List of valid keys
>combining keys

In [101]:
df2.groupby([str.lower, mapping]).mean()

,,data1,data2
key,key,,
a,vowel,1.5,5.0
b,consonant,2.5,4.5
c,consonant,3.5,2.5


# Example

Summarize annual data by decade.

In [103]:
decade = 10 * (planets['year'] // 10)
decade = decade.astype(str) + 's'
decade.name = 'decade'
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0
